In [1]:
import os
import pandas as pd
import zipfile
import pyodbc
import requests
import io
from datetime import datetime, timedelta
import sqlalchemy
import urllib

from TAPI import capturar_fecha

In [2]:
#TODO: iterar en el mismo día 


In [3]:
#definiciones

def iterar_entre_fechas(fecha_desde, fecha_hasta):
    fecha_actual = datetime.strptime(fecha_desde, "%Y-%m-%dT%H:%M:%S.%f%z")
    fecha_fin = datetime.strptime(fecha_hasta, "%Y-%m-%dT%H:%M:%S.%f%z")

    # Asegurarse de que fecha_actual sea exactamente a la medianoche
    fecha_actual = fecha_actual.replace(hour=0, minute=0, second=0, microsecond=0)

    while fecha_actual <= fecha_fin:
        fecha_siguiente = fecha_actual + timedelta(hours=23, minutes=59)
        yield fecha_actual, fecha_siguiente
        # Añadir un día para la próxima iteración
        fecha_actual += timedelta(days=1)

def leer_datos_mdb(mdb_file_path):
    conn_mdb = pyodbc.connect(f"DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={mdb_file_path};")
    cursor_mdb = conn_mdb.cursor()
    query = "SELECT * FROM VALORES_GENERADORES;"
    cursor_mdb.execute(query)
    rows = cursor_mdb.fetchall()
    columns = [column[0] for column in cursor_mdb.description]
    df = pd.DataFrame(rows, columns=columns)
    conn_mdb.close()
    return df

In [4]:
#testing iteracion

fecha_desde = "2023-01-01T00:00:00.000-03:00"
fecha_hasta = "2023-01-03T23:59:00.000-03:00"

for fecha_actual, fecha_siguiente in iterar_entre_fechas(fecha_desde, fecha_hasta):
    #print(fecha_actual.isoformat())
    #print(fecha_siguiente.isoformat())
    dia_mdb = fecha_actual.strftime("%d-%m-%Y")
    print(dia_mdb)

01-01-2023
02-01-2023
03-01-2023


In [6]:
# DATOS DE LA BBDD, SERVER Y TABLA

#server = 'DESKTOP-37ESKFT\SQLEXPRESS'
server = 'DARCCVWSQL19'
database = 'TAPI'
tabla = 'DiarioTest'

# Fechas para seleccionar el día de la carga se debe iterar

fecha_desde = "2023-01-01T00:00:00.000-03:00"
fecha_hasta = "2023-01-02T23:59:59.000-03:00"

#URL para capturar Id del documento y el zip file:

#Defino la tabla de CAMMESA que me voy a traer
NEMO = "PARTE_POST_OPERATIVO_UNIF"
#Busco los zip disponibles para traer así puedo extraer el id
URL = f"https://api.cammesa.com/pub-svc/public/"

method_id = "findDocumentosByNemoRango?" #ID
method_zip = "findAllAttachmentZipByNemoId?"
zip_path = r"C:\Users\Jony\OneDrive - Pampa Energia\Laburo-Estudio\Produccion\Proyectos\Tablero de Análisis de Pérdidas de Ingresos\Proyecto\.zips"
mdb_path = r"C:\Users\Jony\OneDrive - Pampa Energia\Laburo-Estudio\Produccion\Proyectos\Tablero de Análisis de Pérdidas de Ingresos\Proyecto\.zips\.mdb"

connection_string = f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Establecer la conexión con la base de datos de SQL Server
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

url_doc_id = f"{URL}{method_id}fechadesde={fecha_desde}&fechahasta={fecha_hasta}&nemo={NEMO}"

dataframes = []
dfout = pd.DataFrame()

for fecha_actual, fecha_siguiente in iterar_entre_fechas(fecha_desde, fecha_hasta):
    valores_generadores = pd.DataFrame()
    url_doc_id = f"{URL}{method_id}fechadesde={fecha_actual.isoformat()}&fechahasta={fecha_siguiente.isoformat()}&nemo={NEMO}"

    #obtener el doc_id del dia actual (corregido)
    dia_mdb = fecha_actual.strftime("%d-%m-%Y")
    try:
        with requests.get(url_doc_id) as response:
            if response.status_code == 200:
                PPO=response.json()
                doc_id = PPO[-1]['id']
                print(doc_id)
            else:
                print("La solicitud falló con el código de estado:", response.status_code)
    except requests.exceptions.RequestException as e:
        # Manejar la excepción
        print("Error al realizar la solicitud:", e)

    url_zip = f"{URL}{method_zip}docId={doc_id}&nemo={NEMO}"

    #descargar el .zip del doc_id (corregido)

    try:
        with requests.get(url_doc_id) as response:
            if response.status_code == 200:
                r = requests.get(url_zip)

                # Crear un objeto ZipFile a partir del contenido descargado
                z = zipfile.ZipFile(io.BytesIO(r.content))

                # Directorio de destino para extraer los archivos ZIP
                destination_directory = ".zips"

                # Extraer todos los archivos del ZIP en el directorio específico
                z.extractall(destination_directory)
                zip_name = z.namelist()[0]
            else:
                print("La solicitud falló con el código de estado:", response.status_code)

    except requests.exceptions.RequestException as e:
        # Manejar la excepción
        print("Error al realizar la solicitud:", e)
    
    #Colocar los PATHs correctos donde se traeran los archivos
    
    path_zip_dia = f"{zip_path}\{zip_name}"

    display(path_zip_dia)

    
    try:
        # Extrae el archivo MDB de cada archivo ZIP diario
        with zipfile.ZipFile(path_zip_dia, 'r') as zip_ref:
            # Encontrar el nombre del archivo MDB dentro del ZIP diario
            archivo_mdb = os.path.splitext(zip_name)[0] + ".mdb"
            zip_ref.extract(archivo_mdb, path=mdb_path)

   
        # Lee el archivo MDB y cargar la tabla VALORES_GENERADORES en un dataframe
        mdb_file = os.path.join(mdb_path, archivo_mdb)
        conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={mdb_file};"
        conn = pyodbc.connect(conn_str)
        valores_generadores = pd.read_sql("SELECT * FROM VALORES_GENERADORES", conn)
        conn.close()
        valores_generadores.insert(0, 'FECHA', dia_mdb)
        quoted = urllib.parse.quote_plus(connection_string)

        # Por limitaciones de tamaño de excel filtramos solo las máquinas Pampa
        valores_filtrados = ["ADTOHI", "AR21EO", "BAHIEO", "BBLATV29", "BBLATV30",
                            "BBLMDI01", "BBLMDI02", "BBLMDI03", "BBLMDI04", 
                            "BBLMDI05", "BBLMDI06", "CERITV01", "CORTEO", 
                            "EBARTG01", "EBARTG02", "EBARTV01", "ETIGHI", 
                            "GEBATG01", "GEBATG02", "GEBATG03", "GEBATG04", 
                            "GEBATV01", "GEBATV02", "GUEMTG01", "GUEMTV11", 
                            "GUEMTV12", "GUEMTV13", "LDLATG01", "LDLATG02", 
                            "LDLATG03", "LDLATG04", "LDLATG05", "LDLATV01", 
                            "LDLMDI01", "LREYHB", "NIH1HI", "NIH2HI", "NIH3HI", 
                            "PAMEEO", "PEP3EO", "PILBDI01", "PILBDI02", 
                            "PILBDI03", "PILBDI04", "PILBDI05", "PILBDI06", "PIQIDI01", "PPLEHI"]

        # Filtrar el DataFrame por los valores especificados en la columna "GRUPO"
        df_filtrado = valores_generadores[valores_generadores['GRUPO'].isin(valores_filtrados)]        

        engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

        df_filtrado.to_sql('DiarioTest', schema='dbo', con=engine, if_exists='append', chunksize=2000)

        dfout = pd.concat([dfout,df_filtrado], ignore_index=True)

    except FileNotFoundError:
        print(f"El archivo {zip_name} no se encontró. Saltando al siguiente archivo...")

01255BEF763F7DD003258951005A1698


'C:\\Users\\Jony\\OneDrive - Pampa Energia\\Laburo-Estudio\\Produccion\\Proyectos\\Tablero de Análisis de Pérdidas de Ingresos\\Proyecto\\.zips\\PO230101.zip'

El archivo PO230101.zip no se encontró. Saltando al siguiente archivo...
942AA61382F54C3003258951005A7DC5


'C:\\Users\\Jony\\OneDrive - Pampa Energia\\Laburo-Estudio\\Produccion\\Proyectos\\Tablero de Análisis de Pérdidas de Ingresos\\Proyecto\\.zips\\PO230102.zip'

El archivo PO230102.zip no se encontró. Saltando al siguiente archivo...


In [25]:
#SELEC * FROM Diario;

connection_string = f'DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};Trusted_Connection=yes;'

# Conectar a la base de datos
connection = pyodbc.connect(connection_string)

# Crear un cursor para ejecutar consultas
cursor = connection.cursor()

# TODO: Crear acá la tabla en la BBDD
#  Consulta SQL
query = 'SELECT * FROM DiarioTest'

# Ejecutar la consulta
df = pd.read_sql(query, connection)

# Cerrar el cursor
cursor.close()

C:\Users\Jony\AppData\Local\Temp\ipykernel_5712\1236665344.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


In [40]:
dfout = dfout.sort_values('FECHA')


In [41]:
df = df.sort_values('FECHA')

In [42]:
df

,FECHA,GRUPO,HORA,ARL,ESTADO_OPE,PI_SOTR,ARL_ECON,EST_REMUN,ENERGIA,PRECIO_NODO,...,POT_DISP_R19,FACTOR_KM,HORA_REPRESENTATIVA,ENERG_GEN_TRANSAC,ENERG_OPE_TRANSAC,SOBREC_FORZ_PROPIO,PGENE_RES354,CGU_PROPIO,CGU_MEM,CGU_GAS_CEDIDO
0,01-01-2023,9JULDI02,3,1,i-i,0.0,1,i-i,0.000000,1682.0,...,0.000,1.0,None,NaN,NaN,None,NaN,NaN,NaN,NaN
10666,01-01-2023,PNUETV08,22,1,Arr,0.0,1,Arr,0.000000,1682.0,...,0.000,1.0,None,NaN,NaN,None,NaN,NaN,NaN,NaN
10667,01-01-2023,PNUETV08,23,1,Arr,0.0,1,Arr,0.000000,1682.0,...,0.000,1.0,None,NaN,NaN,None,NaN,NaN,NaN,NaN
10668,01-01-2023,PNUETV08,24,1,Arr,0.0,1,Arr,0.000000,1682.0,...,0.000,1.0,None,NaN,NaN,None,NaN,NaN,NaN,NaN
10669,01-01-2023,PNUETV09,1,1,===,238.0,1,===,223.876007,1682.0,...,237.700,1.0,S,NaN,NaN,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20732,02-01-2023,GEBATG03,12,1,CMI,159.0,1,===,137.748001,1682.0,...,165.000,1.0,None,NaN,NaN,None,165.000,4673.61,4673.61,4673.61
20733,02-01-2023,GEBATG03,13,1,===,159.0,1,===,150.848007,1682.0,...,165.000,1.0,S,NaN,NaN,None,165.000,4539.91,4539.91,4539.91
20734,02-01-2023,GEBATG03,14,1,===,158.0,1,===,148.850998,1682.0,...,163.736,1.0,S,NaN,NaN,None,163.736,4570.55,4570.55,4570.55
20736,02-01-2023,GEBATG03,16,1,===,159.0,1,===,150.201004,1682.0,...,165.000,1.0,S,NaN,NaN,None,165.000,4581.43,4581.43,4581.43


In [43]:
dfout

,FECHA,GRUPO,HORA,ARL,ESTADO_OPE,PI_SOTR,ARL_ECON,EST_REMUN,ENERGIA,PRECIO_NODO,...,POT_DISP_R19,FACTOR_KM,HORA_REPRESENTATIVA,ENERG_GEN_TRANSAC,ENERG_OPE_TRANSAC,SOBREC_FORZ_PROPIO,PGENE_RES354,CGU_PROPIO,CGU_MEM,CGU_GAS_CEDIDO
0,01-01-2023,9JULDI01,1,1,lib,0.0,1,lib,0.000,1682.0,...,4.200,1.0,None,NaN,NaN,None,NaN,NaN,NaN,NaN
10632,01-01-2023,PERGDI01,1,1,===,0.0,1,===,2.368,1682.0,...,2.368,1.0,S,NaN,NaN,None,NaN,NaN,NaN,NaN
10633,01-01-2023,PERGDI01,2,1,===,0.0,1,===,2.368,1682.0,...,2.368,1.0,S,NaN,NaN,None,NaN,NaN,NaN,NaN
10634,01-01-2023,PERGDI01,3,1,===,0.0,1,===,2.368,1682.0,...,2.368,1.0,S,NaN,NaN,None,NaN,NaN,NaN,NaN
10635,01-01-2023,PERGDI01,4,1,===,0.0,1,===,2.368,1682.0,...,2.368,1.0,S,NaN,NaN,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21284,02-01-2023,FRIATG01,21,1,I-K,0.0,1,I-K,0.000,1682.0,...,58.980,1.0,None,NaN,NaN,None,NaN,NaN,NaN,NaN
21285,02-01-2023,FRIATG01,22,1,I-K,0.0,1,I-K,0.000,1682.0,...,58.980,1.0,None,NaN,NaN,None,NaN,NaN,NaN,NaN
21286,02-01-2023,FRIATG01,23,1,I-K,0.0,1,I-K,0.000,1682.0,...,58.980,1.0,None,NaN,NaN,None,NaN,NaN,NaN,NaN
21273,02-01-2023,FRIATG01,10,1,I-K,0.0,1,I-K,0.000,1682.0,...,58.980,1.0,None,NaN,NaN,None,NaN,NaN,NaN,NaN


In [31]:
dfout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31920 entries, 0 to 31919
Data columns (total 40 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   FECHA                31920 non-null  object 
 1   GRUPO                31920 non-null  object 
 2   HORA                 31920 non-null  int64  
 3   ARL                  31920 non-null  int64  
 4   ESTADO_OPE           31920 non-null  object 
 5   PI_SOTR              31920 non-null  float64
 6   ARL_ECON             31920 non-null  int64  
 7   EST_REMUN            31920 non-null  object 
 8   ENERGIA              31920 non-null  float64
 9   PRECIO_NODO          31920 non-null  float64
 10  PR_REM_ENERGIA       18504 non-null  float64
 11  SMEC                 31920 non-null  object 
 12  POT_DISP             31920 non-null  float64
 13  POT_OPE_CPO          27024 non-null  float64
 14  COMPRA_SPOT          30336 non-null  float64
 15  SCTD                 17125 non-null 

In [34]:
dif = df.compare(dfout)

In [35]:
dif

FECHA                 GRUPO            HORA       ESTADO_OPE  \
             self       other      self     other  self other       self   
0             NaN         NaN  9JULDI02  9JULDI01   3.0   1.0        i-i   
1             NaN         NaN  9JULDI02  9JULDI01   4.0   2.0        i-i   
2             NaN         NaN  9JULDI02  9JULDI01   5.0   3.0        i-i   
3             NaN         NaN  9JULDI02  9JULDI01   6.0   4.0        i-i   
4             NaN         NaN  9JULDI02  9JULDI01   7.0   5.0        i-i   
...           ...         ...       ...       ...   ...   ...        ...   
31915  01-01-2023  02-01-2023    ENE1EO  ZARATG04   7.0  20.0        ===   
31916  01-01-2023  02-01-2023    ENE1EO  ZARATG04   8.0  21.0        ===   
31917  01-01-2023  02-01-2023    ENE1EO  ZARATG04   9.0  22.0        ===   
31918  01-01-2023  02-01-2023    ENE1EO  ZARATG04  10.0  23.0        ===   
31919  01-01-2023  02-01-2023    ENE1EO  ZARATG04  11.0  24.0        ===   

            PI_SOTR        ... ENERG_OPE_TRANSAC       PGENE_RES354        \
      other    self other  ...              self other         self other   
0       lib     NaN   NaN  ...               NaN   NaN          NaN   NaN   
1       lib     NaN   NaN  ...               NaN   NaN          NaN   NaN   
2       lib     NaN   NaN  ...               NaN   NaN          NaN   NaN   
3       lib     NaN   NaN  ...               NaN   NaN          NaN   NaN   
4       lib     NaN   NaN  ...               NaN   NaN          NaN   NaN   
...     ...     ...   ...  ...               ...   ...          ...   ...   
31915   I-K     NaN   NaN  ...               NaN   NaN          NaN   NaN   
31916   I-K     NaN   NaN  ...               NaN   NaN          NaN   NaN   
31917   I-K     NaN   NaN  ...               NaN   NaN          NaN   NaN   
31918   I-K     NaN   NaN  ...               NaN   NaN          NaN   NaN   
31919   I-K     NaN   NaN  ...               NaN   NaN          NaN   NaN   

      CGU_PROPIO       CGU_MEM       CGU_GAS_CEDIDO        
            self other    self other           self other  
0            NaN   NaN     NaN   NaN            NaN   NaN  
1            NaN   NaN     NaN   NaN            NaN   NaN  
2            NaN   NaN     NaN   NaN            NaN   NaN  
3            NaN   NaN     NaN   NaN            NaN   NaN  
4            NaN   NaN     NaN   NaN            NaN   NaN  
...          ...   ...     ...   ...            ...   ...  
31915        NaN   NaN     NaN   NaN            NaN   NaN  
31916        NaN   NaN     NaN   NaN            NaN   NaN  
31917        NaN   NaN     NaN   NaN            NaN   NaN  
31918        NaN   NaN     NaN   NaN            NaN   NaN  
31919        NaN   NaN     NaN   NaN            NaN   NaN  

[31920 rows x 66 columns]

In [20]:
quoted = urllib.parse.quote_plus(connection_string)
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
dfout.to_sql('DiarioTest', schema='dbo', con=engine, if_exists='append')

15